<a href="https://colab.research.google.com/github/anna-marshalova/methods-and-algorithms-of-computational-linguistics/blob/main/feature_extraction_(task_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Алгоритм выделения признаков текстов для классификации тональности твита на данных соревнования SentiRuEval-2016.

In [ ]:
from lxml import etree
import csv
import re

In [ ]:
from typing import List, Tuple

In [ ]:
pip install spacy_udpipe

     |████████████████████████████████| 304 kB 40.7 MB/s 
     |████████████████████████████████| 5.9 MB 29.1 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 456 kB 57.9 MB/s 
     |████████████████████████████████| 10.1 MB 41.5 MB/s 
     |████████████████████████████████| 623 kB 51.5 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626610 sha256=675e2332c335e7c41e1b45915298078f57d599633e4182f97522e175c532ca4f
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thin

In [ ]:
import spacy_udpipe

spacy_udpipe.download("ru") 
nlp = spacy_udpipe.load("ru")

Downloaded pre-trained UDPipe model for 'ru' language


Загружаем словарь оценочных слов и выражений русского языка [РуСентиЛекс](http://www.labinform.ru/pub/rusentilex/index.htm)

In [ ]:
def load_emotions(filename):
  with open("emo_dict.csv", encoding="utf-8") as file:
    all_words = csv.DictReader(file)
    positive_list=[]
    negative_list=[]
    neutral_list=[]
    for row in all_words:
                row = list(row.values())[0].split(';')
                if 'PSTV' in row:
                    positive_list.append(row[0])
                elif 'NGTV' in row:
                    negative_list.append(row[0])
                elif 'NEUT' in row:
                    neutral_list.append(row[0])
  return positive_list,negative_list,neutral_list

In [ ]:
positive_list,negative_list,neutral_list=load_emotions('emo_dict.csv')

Загружаем [список](https://habr.com/ru/post/21949/) стран, городов и регионов

In [ ]:
 def load_places(file_name):
  places=[]
  with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
        root = etree.fromstring(xml_data)
        for place in root.getchildren():
              for row in place.getchildren():
                if row.tag=='name':
                    places.append(row.text.lower())
  return places

In [ ]:
places=load_places('rocid.xml')

In [ ]:
extract=[('telegram',re.compile('(http[s]?:)?\/\/t\.co')),('hashtag',re.compile('#.?'))]
delete=[('number',re.compile('[+-]?[0-9\-\.,]+[%]?')), ('url', re.compile('http')),('mention',re.compile('@.+')),('e-mail', re.compile('.+@.+\.')),('hashtag',re.compile('#.?'))]

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class MyVectorizer(BaseEstimator,TransformerMixin):
  def __init__(self,texts,vectors):
    self.texts=texts
    self.vectors=vectors
    self.features=['telegram','hashtag','geo','positive','negative','neutral','exclamation']
  def fit(self, X, y=None):
    return self
  def transform(self, X, y=None):
    return self.vectors
  def get_feature_names(self):
    return self.features

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [ ]:
vectorizer = CountVectorizer(lowercase=True, min_df=3)

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2

In [ ]:
selector = SelectPercentile(chi2, percentile=20)

In [ ]:
from sklearn.pipeline import FeatureUnion

In [ ]:
class FeatureExtractor:
  def __init__(self,file):
    self.texts, self.vectors,self.labels=self.load_sentirueval_2016(file)
    print('loaded successfully')
    vectorizer.fit(self.texts)
    self.X=vectorizer.transform(self.texts)
    print('transformed successfully')
    tfidf=TfidfTransformer()
    myvec=MyVectorizer(self.texts,self.vectors)
    transformer =FeatureUnion([('tfidf',tfidf),('myvec',myvec)])
    transformer.fit(self.X)
    self.X_transformed = transformer.transform(self.X)
    print('binary features added successfully')
    # как я поняла, idf нужны для сортировки, поэтому добавленным признакам можно сопоставить любое число. я выбрала 10, чтобы эти признаки оказались в начале списка
    numbers=[10 for i in enumerate(myvec.get_feature_names())]
    idfs=[idf for idf in tfidf.idf_]+numbers
    self.tokens_with_IDF = list(zip(vectorizer.get_feature_names()+myvec.get_feature_names(), idfs))
    selector.fit(self.X_transformed, self.labels)
    self.selected_tokens_with_IDF = [self.tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]
    print('selected successfully')
    self.sorted_selected_tokens_with_IDF=sorted(self.selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))
    print('sorted successfully')

  def load_sentirueval_2016(self,file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    vectors=[]
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        #лемматизация
                        new_text,vector = self.lemmatize(str(column.text).strip())
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                vectors.append(vector)
                labels.append(new_label)
            break
    return texts, vectors,labels

  def lemmatize(self,text):
   text_lemmatized=[]
   vector=[]
   geo=0
   positive=0
   negative=0
   neutral=0
   excl=0
   doc = nlp(text.lower())
   #добавляем бинарные признаки наличия ссылки на телеграм-канал и хештега
   for element in extract:
       if element[1].search(text):
         vector.append(1)
       else:
         vector.append(0)
   for token in doc:
     lemma=token.lemma_
     #удаляем геграфические названия и записываем их наличие
     if lemma in places:
       geo=1
       lemma=''
    #добавляем бинарные признаки наличия эмоциональных слов
     if lemma in positive_list:
      positive=1
     if lemma in negative_list:
      negative=1
     if lemma in neutral_list:
      neutral=1
    #добавляем бинарный признак наличия восклицательного знака
     if lemma == '!':
       excl=1
     #удаляем лишнее
     for element in delete:
       if element[1].match(lemma):
         lemma=''
     #удаляем элементы, для которых часть речи не определилась
     if token.pos_=='X':
       lemma=''
     text_lemmatized.append(lemma)
   vector.extend([geo,positive,negative,neutral,excl])
   return  ' '.join(text_lemmatized),vector
  

In [ ]:
def print_features(features):
    for feature, idf in features [0:40]: print('{0:.6f} => {1}'.format(idf, feature))

In [ ]:
banks=FeatureExtractor('bank_train_2016.xml')

loaded successfully
transformed successfully
binary features added successfully
selected successfully
sorted successfully


In [ ]:
print_features(banks.sorted_selected_tokens_with_IDF)

10.000000 => exclamation
10.000000 => hashtag
10.000000 => positive
10.000000 => telegram
8.761426 => ауэрбанка
8.761426 => бум
8.761426 => бурятие
8.761426 => внедрение
8.761426 => вносить
8.761426 => выбирать
8.761426 => говно
8.761426 => догонять
8.761426 => компание
8.761426 => любимый
8.761426 => миллионер
8.761426 => навстретить
8.761426 => навязывать
8.761426 => неустойка
8.761426 => ок
8.761426 => олы
8.761426 => оперативно
8.761426 => популярность
8.761426 => стабильно
8.761426 => супер
8.761426 => тысячный
8.761426 => убить
8.761426 => уэк
8.761426 => фон
8.761426 => часы
8.761426 => школьный
8.761426 => юникорбанка
8.538282 => бренд
8.538282 => введёть
8.538282 => воровство
8.538282 => впервые
8.538282 => гореть
8.538282 => горь
8.538282 => заморозить
8.538282 => идиот
8.538282 => красивый


In [ ]:
tkks=FeatureExtractor('tkk_train_2016.xml')

loaded successfully
transformed successfully
binary features added successfully
selected successfully
sorted successfully


In [ ]:
print_features(tkks.sorted_selected_tokens_with_IDF)

10.000000 => exclamation
10.000000 => hashtag
10.000000 => positive
10.000000 => telegram
8.678326 => виваселла
8.678326 => внедрять
8.678326 => вэб
8.678326 => дзержинска
8.678326 => долой
8.678326 => емкость
8.678326 => заботиться
8.678326 => защищать
8.678326 => инноватор
8.678326 => магазине
8.678326 => мичуринска
8.678326 => настроить
8.678326 => оборудовать
8.678326 => перерыв
8.678326 => поболтать
8.678326 => подарочка
8.678326 => поезд
8.678326 => развернуть
8.678326 => сверхскоростный
8.678326 => совершать
8.678326 => сёдня
8.678326 => устраивать
8.678326 => цепь
8.678326 => шикарный
8.455183 => ddos
8.455183 => акселерационный
8.455183 => атаковать
8.455183 => бля
8.455183 => вгтрк
8.455183 => видеоконференция
8.455183 => волс
8.455183 => выручить
8.455183 => гребанный
8.455183 => доход
8.455183 => европейский
8.455183 => задействовать


Таким образом, наличие географических названий, нейтральных и негативных слов оказались незначимыми признаками. Возможно, это потому, что геграфические названия могут встретиться в любых твитах (а не только нейтральных, как я предпологала), а подгружаемый словарь эмоциональных слов основан на ассоциациях. Допустим, слово "аборт" в нем отмечено как негативное, потому что вызывает негативные ассоциации, но встретится оно, скорее, в нейтральном твите. 

А наличие положительных прилагательных, ссылки на телеграм канал, хештега и восклицательного знака в обоих случаях значимы. Вероятно, телеграм-ссылка - признак нейтральных твитов, а хештег и восклицательный знак чаще встречаются в эмоциональных твитах (причем, скорее всего, положительных). 

Еще, возможно, некоторые слова оказались значимыми признаками из-за странной лемматизации. Например, "льзуются" - это, скорее всего, лемматизированная часть слова "пользуются", которое в твите, вероятно, было написано раздельно.



